# This notebook compares ReAct with One Big Prompt

there are three variations to test
* one big prompt
* ReAct using RAG as the search function
* ReAct with search function that returns the whole PDF (Part.from_data('pdf'))

In [70]:
import sys
import os
import importlib
import json
from vertexai.generative_models import GenerationConfig, GenerativeModel, Content, Part  
sys.path.append(os.path.abspath('../utils'))
from rich.markdown import Markdown as rich_Markdown

# convert tool
import tool_functions
importlib.reload(tool_functions)
from tool_functions import convert_to_tool # convert self-defined functions to Tool objects

# RAG class
import rag
importlib.reload(rag)
from rag import RAG # RAG search function

# doc_ai chunks
import doc_ai
importlib.reload(doc_ai)
from doc_ai import read_jsonl_to_json_list

# ReAct Agent
import react_agent
importlib.reload(react_agent)
from react_agent import ReactAgent

# one big prompt Agent
import big_prompt_agent
importlib.reload(big_prompt_agent)
from big_prompt_agent import BigPromptAgent

# 1. naf23.pdf

In [2]:
pdf_path='../data/naf23.pdf'

## 1.1 initialize agents, and test them on a simple question

### 1.1.1 one big prompt agent

In [24]:
agent_one_big_prompt=BigPromptAgent(pdf_path=pdf_path)
query="what is National Ataxia Foundation's total revenue in 2023?"


In [25]:
agent_one_big_prompt=BigPromptAgent(pdf_path=pdf_path)
answer = agent_one_big_prompt.run(query=query)
print(answer)

The National Ataxia Foundation's total revenue in 2023 was $4,184,787. This is the sum of total support ($3,205,136) and total revenue ($979,651).  It also includes the net assets released from restrictions, which offset each other ($975,954 and -$975,954). This information is found on page 7 of the provided financial report.


### 1.1.2 ReAct agent with RAG

In [3]:
prefix='react_data/output/6845398121975108687/0'
with open('../config.json', 'r') as file:
    ids = json.load(file)
bucket_name = ids['bucket_name']

chunk_json_list=read_jsonl_to_json_list(bucket_name, prefix)
text_chunks=[]
for json_obj in chunk_json_list:
    text_chunks.append(json_obj['content'])

current file name: react_data/output/6845398121975108687/0/naf23-0_ChunkSize200.jsonl


In [7]:
# character length of each chunk
for chunk in text_chunks:
    print(len(chunk), end=' ')

856 727 625 821 1230 1141 512 1031 435 101 1024 355 113 1033 431 952 821 127 979 661 101 1078 154 424 1200 840 1228 585 1261 970 1343 1063 1289 757 1257 1214 920 1163 1018 773 837 1162 648 1188 1035 991 985 963 1129 1110 854 1009 1149 

In [39]:
data_path='../data/naf23.pdf'
rag_instance=RAG(pdf_path= data_path, chunking_method='N/A', input_chunks=text_chunks)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [40]:
def search1(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base.
    You can use this search function to search for financial information of specific companies you do not know. 
    Treat this function as a internal wiki search that you can use.

    Args:
        query (str): input query            
    """
    results = rag_instance.search(query, method='ensemble', num_top_chunks=3)
    return json.dumps({"relevant information in the auditor notes": results})


# convert this search function to a Tool object
search_tool_rag_chunks = convert_to_tool(search1)

In [71]:
agent_react = ReactAgent(tools=[search_tool_rag_chunks])

In [42]:
agent_react.run(query="what is the total revenue of National Ataxia Foundation in 2023?")

---------------------------------------- iteration 0  ----------------------------------------


---------------------------------------- iteration 1  ----------------------------------------


"The National Ataxia Foundation's total revenue in 2023 was $979,651."

### 1.1.3 ReAct agent that returns the full pdf

In [26]:
auditor_notes_doc=None
with open(pdf_path, 'rb') as fp:
    auditor_notes_doc=Part.from_data(data=fp.read(),mime_type='application/pdf')

In [27]:
# define a search function that returns the entire pdf read from Part.from_data()
def search2(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    return auditor_notes_doc

# convert this search function to a Tool object
search_tool_rag_full_pdf = convert_to_tool(search2)

In [72]:
agent_react_full_pdf = ReactAgent(tools=[search_tool_rag_full_pdf])

In [29]:
# make sure they have different tools
print(agent_react.tools)
print(agent_react_full_pdf.tools)

In [30]:
query

"what is National Ataxia Foundation's total revenue in 2023?"

In [31]:
agent_react_full_pdf.run(query=query)

---------------------------------------- iteration 0  ----------------------------------------


---------------------------------------- iteration 1  ----------------------------------------


"The National Ataxia Foundation's total revenue in 2023 was $4,184,787."

### 1.2 question 1

In [60]:
query="What factors contributed to the decrease of cash and cash equivalents from $1,969,164 in 2022 to $1,008,716 in 2023 for National Ataxia Foundation"

In [52]:
rich_Markdown(agent_one_big_prompt.run(query=query))

The Statement of Cash Flows shows that the decrease in cash and cash equivalents from 2022 to 2023 was $960,448.   
This was driven by a net decrease in cash from operating activities of $955,522 and a net decrease in cash from    
investing activities of $4,926.  The decrease in cash from operating activities was primarily due to a decrease in 
net assets of $811,671.  The decrease in cash from investing activities was due to the purchase of investments     
exceeding the proceeds from the sale of investments.

In [61]:
agent_react.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------


'The decrease in cash and cash equivalents from $1,969,164 in 2022 to $1,008,716 in 2023 for the National Ataxia Foundation is primarily due to a significant negative cash flow from operating activities of $(955,522) in 2023, compared to a positive cash flow of $562,069 in 2022.  This shift is driven by a decrease in net assets and unfavorable changes in working capital (increases in accounts receivable and decreases in accounts payable and deferred revenue).  While investment activities had a small negative impact, the primary driver of the decrease was the substantial swing in operating cash flow.'

In [62]:
agent_react_full_pdf.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------


'The decrease in cash and cash equivalents from $1,969,164 in 2022 to $1,008,716 in 2023 for the National Ataxia Foundation was primarily due to a net cash outflow from investing activities, mainly driven by investment purchases exceeding sales proceeds.  Operating activities also contributed to the decrease, using $955,522 in cash during 2023, compared to providing $562,069 in 2022.'

* <span style="color: blue; font-weight: bold;"> All method fail to go to check or mention specifically `"Note 10: Liquidity and Availability of Financial Assets"` where `The Foundation invests cash in excess of monthly requirements in short-term investments such as money market funds, certificates of deposit, savings accounts, or other similar investments`</span>

### 1.3 question 2

In [65]:
query="Why did deferred revenue increase significantly in 2023 for National Ataxia Foundation?"

In [57]:
agent_one_big_prompt.run(query=query)

"The auditor's report shows that deferred revenue related to the Drug Development Collaborative increased from $0 in 2022 to $100,000 in 2023.  Deferred conference revenue was $9,810 in 2023 and $22,290 in 2022.  Therefore, the significant increase in deferred revenue is attributable to the Drug Development Collaborative."

In [66]:
agent_react.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------


'The significant increase in deferred revenue for the National Ataxia Foundation in 2023 is primarily due to a new Drug Development Collaborative, which generated $100,000 in deferred revenue.  This increase was offset by a decrease in deferred conference revenue.'

In [67]:
agent_react_full_pdf.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------


'The significant increase in deferred revenue for the National Ataxia Foundation in 2023 is primarily attributable to a $100,000 increase in payments received in advance for the Drug Development Collaborative. There was also a smaller increase related to conference revenue received in advance.'

* <span style="color: blue; font-weight: bold;"> All correct </span>
* <span style="color: blue; font-weight: bold;"> ReAct just use 1 iteration </span>
* <span style="color: blue; font-weight: bold;"> ReAct with full pdf breaks the prompt specification by outputing `<observation>` to us </span>

### 1.4 question 3

In [68]:
query="What contributed to the significant increase in net assets with donor restrictions from 2022 to 2023 for National Ataxia Foundation"

In [73]:
agent_one_big_prompt.run(query=query)

"The auditor's report does not provide a specific reason for the increase in net assets with donor restrictions.  It does, however, show the net assets with donor restrictions increased by $638,761 from 2022 to 2023.  The Statement of Activities shows this increase was due to contributions exceeding expenses.  Specifically, contributions with donor restrictions totaled $1,614,715, while expenses totaled $811,671.  The notes further detail the various funds that hold the donor-restricted net assets."

In [74]:
agent_react.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------


"The National Ataxia Foundation's net assets with donor restrictions increased significantly from $2,041,806 in 2022 to $2,680,567 in 2023.  This increase is primarily due to new contributions to several funds, most notably the SCA2 Fund (increase of $380,250) and the establishment of the MR Imaging Study Fund ($612,044). While some funds saw decreases (like the CSO Funding and SCA3 Clementz Fund), the new contributions and the new MR Imaging Study Fund more than offset these decreases, leading to the overall increase in net assets with donor restrictions."

In [76]:
agent_react_full_pdf.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------



---------------------------------------- iteration 2  ----------------------------------------


'The net assets with donor restrictions increased by $638,761 from 2022 to 2023. This is due to contributions of $1,614,715 with donor restrictions and releases from restrictions of $975,954 during 2023.'

* <span style="color: blue; font-weight: bold;"> big prompt cannot find the answer </span>
* <span style="color: blue; font-weight: bold;"> ReAct with RAG find the answer in one iteration </span>
* <span style="color: blue; font-weight: bold;"> ReAct with full pdf does not find the detailed answer, which is listed in `"as detailed in Note 6 of the financial statements."` </span>

<font color='red'> <b>when RAG with ReAct finds the answer in one iteration, it is basically just a big prompt using RAG </b></font>

### 1.5 question 4

In [77]:
query="What caused the significant decrease in the long-term operating lease liability from 2022 to 2023 for National Ataxia Foundation?"

In [78]:
agent_one_big_prompt.run(query=query)

"The provided auditor's report does not contain information regarding operating leases or the reason for the decrease in long-term operating lease liability.  Therefore, I cannot answer your question."

In [80]:
agent_react.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------


'The long-term operating lease liability for the National Ataxia Foundation *decreased* by $31,549 from 2022 to 2023. This is a normal consequence of lease accounting, where the liability is reduced over time as lease payments are made and the right-of-use (ROU) asset is amortized.  The provided financial statement excerpt confirms this decrease as part of the cash flow statement.'

In [81]:
agent_react_full_pdf.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------



---------------------------------------- iteration 2  ----------------------------------------


'The decrease in the long-term operating lease liability from 2022 to 2023 for the National Ataxia Foundation is primarily due to the amortization of the lease liability over time as per their agreement with Interchange Investors, LLC.  The lease has fixed monthly payments that increase incrementally until expiration in September 2025. As payments are made, the outstanding liability decreases.'

* <span style="color: blue; font-weight: bold;">one big prompt cannot answer</span>
* <span style="color: blue; font-weight: bold;">ReAct with RAG got the answer in one iteration</span>
* <span style="color: blue; font-weight: bold;">ReAct with full PDF used 2 iteration, it got the answer in iteration 1, it provides more detailed explanation in iteration 2</span>


# iccrom.pdf

In [91]:
pdf_path='../data/stc.pdf'

In [92]:
# -------------------------------one big prompt agent --------------------------------
agent_one_big_prompt=BigPromptAgent(pdf_path=pdf_path)

# -------------------------------ReAct agent with RAG --------------------------------
prefix='react_data/output/8630714554969055591/0'
with open('../config.json', 'r') as file:
    ids = json.load(file)
bucket_name = ids['bucket_name']

chunk_json_list=read_jsonl_to_json_list(bucket_name, prefix)
text_chunks=[]
for json_obj in chunk_json_list:
    text_chunks.append(json_obj['content'])


for chunk in text_chunks:
    print(len(chunk), end=' ')

rag_instance=RAG(pdf_path= pdf_path, chunking_method='N/A', input_chunks=text_chunks)


def search1(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base.
    You can use this search function to search for financial information of specific companies you do not know. 
    Treat this function as a internal wiki search that you can use.

    Args:
        query (str): input query            
    """
    results = rag_instance.search(query, method='ensemble', num_top_chunks=3)
    return json.dumps({"relevant information in the auditor notes": results})


search_tool_rag_chunks = convert_to_tool(search1)
agent_react = ReactAgent(tools=[search_tool_rag_chunks])


# -------------------------------ReAct agent with full pdf ---------------------------
auditor_notes_doc=None
with open(pdf_path, 'rb') as fp:
    auditor_notes_doc=Part.from_data(data=fp.read(),mime_type='application/pdf')
    
def search2(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    return auditor_notes_doc

search_tool_rag_full_pdf = convert_to_tool(search2)
agent_react_full_pdf = ReactAgent(tools=[search_tool_rag_full_pdf])

current file name: react_data/output/8630714554969055591/0/stc-0_ChunkSize200.jsonl
52 1017 545 1121 714 1190 431 949 1158 1021 773 707 1261 1209 533 1265 870 768 1111 1238 446 913 964 1149 194 519 901 694 1047 828 928 1026 654 1029 445 849 1107 1018 607 927 937 1163 1228 560 1193 788 1101 1106 119 

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### question 1

In [93]:
query='Assets changed from €3,448,770.20 in 2020 to €1,526,209.73 in 2021 for Tansport Community. What are the drivers behind this change?'

In [94]:
agent_one_big_prompt.run(query=query)

'Total assets decreased from €3,448,770.20 in 2020 to €1,526,209.73 in 2021. This was driven by a decrease in current assets, specifically cash and cash equivalents, which decreased from €3,254,771.91 to €1,492,643.83.  While non-current assets increased slightly, this was not enough to offset the decrease in current assets.  The notes do not provide further details on the drivers behind the decrease in cash and cash equivalents.'

In [96]:
agent_react.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------



---------------------------------------- iteration 2  ----------------------------------------


'The significant decrease in assets from €3,448,770.20 in 2020 to €1,526,209.73 in 2021 for the Transport Community is primarily due to a reclassification of unused budget appropriations from assets to liabilities in 2021.  The tangible asset portion remained relatively stable, with a slight decrease due to normal depreciation.'

In [99]:
agent_react_full_pdf.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------



---------------------------------------- iteration 2  ----------------------------------------



---------------------------------------- iteration 3  ----------------------------------------


"The decrease in Transport Community's assets from €3,448,770.20 in 2020 to €1,526,209.73 in 2021 is primarily due to a significant increase in unused budget appropriations (€737,686.05 in 2021 vs. €3,254,632.46 in 2020). These unused appropriations, largely from the European Union's contribution, are earmarked for repayment or to be carried forward to the following year's budget.  While operational spending occurred as expected, the large amount of unused funds significantly impacted the available cash and cash equivalents, resulting in the overall decrease in assets.  This suggests a budgeting or funding allocation issue rather than overspending."

* <span style="color: blue; font-weight: bold;">one big prompt cannot answer<span>
* <span style="color: blue; font-weight: bold;">ReAct with RAG found the answer:  decrease in unused appropriations <span>
* <span style="color: blue; font-weight: bold;">ReAct with full pdf also found it<span>
    
* <span style="color: blue; font-weight: bold;">the reason found by chatgpt does not seem to be correct</span>

### question 2

In [100]:
query="Total expenses increased significantly in 2021 compared to 2020. What are the main reasons behind this increase?"


In [101]:
agent_one_big_prompt.run(query=query)

'Total expenses in 2021 amounted to €1,744,964.57, compared to €821,384.63 in 2020. The main drivers for this increase are:\n\n* **Staff costs:** Increased from €710,717.99 in 2020 to €1,385,839.57 in 2021. This is due to the Permanent Secretariat being fully staffed with 21 employed officials in 2021 (with the exception of January), compared to partial staffing in 2020 (Note 9).\n* **Operating expenses:** Increased from €110,666.64 in 2020 to €359,125.00 in 2021. The main driver here is the increase in running costs, which includes office costs, equipment, software, and other costs and services (Note 9 and 16).  Specifically, office costs, equipment, and software increased from €4,343.70 to €10,859.64, and other costs and services increased from €72,082.38 to €114,139.18.  Recruitment costs also contributed to the increase, amounting to €18,481.07 in 2021, compared to €0 in 2020.\n* **Depreciation and amortization:** Increased from €16,136.92 to €13,699.88. This is due to increased in

In [103]:
rich_Markdown('Total expenses in 2021 amounted to €1,744,964.57, compared to €821,384.63 in 2020. The main drivers for this increase are:\n\n* **Staff costs:** Increased from €710,717.99 in 2020 to €1,385,839.57 in 2021. This is due to the Permanent Secretariat being fully staffed with 21 employed officials in 2021 (with the exception of January), compared to partial staffing in 2020 (Note 9).\n* **Operating expenses:** Increased from €110,666.64 in 2020 to €359,125.00 in 2021. The main driver here is the increase in running costs, which includes office costs, equipment, software, and other costs and services (Note 9 and 16).  Specifically, office costs, equipment, and software increased from €4,343.70 to €10,859.64, and other costs and services increased from €72,082.38 to €114,139.18.  Recruitment costs also contributed to the increase, amounting to €18,481.07 in 2021, compared to €0 in 2020.\n* **Depreciation and amortization:** Increased from €16,136.92 to €13,699.88. This is due to increased investment in tangible and intangible assets in 2021 (Notes 3 and 4).\n\n\nIt is important to note that the financial statements were prepared on an accrual basis, meaning expenses are recognized when a transaction occurs, not when payment is made.  This explains why total expenses equal total revenue, resulting in a net surplus of €0.00.\n')

Total expenses in 2021 amounted to €1,744,964.57, compared to €821,384.63 in 2020. The main drivers for this       
increase are:                                                                                                      

 • Staff costs: Increased from €710,717.99 in 2020 to €1,385,839.57 in 2021. This is due to the Permanent          
   Secretariat being fully staffed with 21 employed officials in 2021 (with the exception of January), compared to 
   partial staffing in 2020 (Note 9).                                                                              
 • Operating expenses: Increased from €110,666.64 in 2020 to €359,125.00 in 2021. The main driver here is the      
   increase in running costs, which includes office costs, equipment, software, and other costs and services (Note 
   9 and 16).  Specifically, office costs, equipment, and software increased from €4,343.70 to €10,859.64, and     
   other costs and services increased from €72,082.38 to €114,139.18.  Recruitment costs also contributed to the   
   increase, amounting to €18,481.07 in 2021, compared to €0 in 2020.                                              
 • Depreciation and amortization: Increased from €16,136.92 to €13,699.88. This is due to increased investment in  
   tangible and intangible assets in 2021 (Notes 3 and 4).                                                         

It is important to note that the financial statements were prepared on an accrual basis, meaning expenses are      
recognized when a transaction occurs, not when payment is made.  This explains why total expenses equal total      
revenue, resulting in a net surplus of €0.00.

In [104]:
agent_react.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------



---------------------------------------- iteration 2  ----------------------------------------



---------------------------------------- iteration 3  ----------------------------------------



---------------------------------------- iteration 4  ----------------------------------------



---------------------------------------- iteration 5  ----------------------------------------



---------------------------------------- iteration 6  ----------------------------------------


"Total expenses increased significantly from 821,384.63 EUR in 2020 to 1,744,964.57 EUR in 2021.  A major driver of this increase is staff costs, rising from 710,717.99 EUR in 2020 to 1,385,839.57 EUR in 2021.  Further analysis of the operating expenses reveals the following changes:\n\n* **Running costs:** Increased from 87,091.87 EUR in 2020 to 350,390.24 EUR in 2021.\n* **Recruitment costs:** Increased from 18,481.07 EUR in 2020 to an undisclosed amount in 2021 (likely included in staff costs or other operating expenses).\n* **Finance costs:** Increased from 5,093.70 EUR in 2020 to 8,734.76 EUR in 2021.\n\nThe substantial increase in staff and running costs are the primary contributors to the overall rise in total expenses from 2020 to 2021.  While the provided data doesn't explain *why* these costs increased, it clearly identifies them as the main drivers."

In [105]:
rich_Markdown("Total expenses increased significantly from 821,384.63 EUR in 2020 to 1,744,964.57 EUR in 2021.  A major driver of this increase is staff costs, rising from 710,717.99 EUR in 2020 to 1,385,839.57 EUR in 2021.  Further analysis of the operating expenses reveals the following changes:\n\n* **Running costs:** Increased from 87,091.87 EUR in 2020 to 350,390.24 EUR in 2021.\n* **Recruitment costs:** Increased from 18,481.07 EUR in 2020 to an undisclosed amount in 2021 (likely included in staff costs or other operating expenses).\n* **Finance costs:** Increased from 5,093.70 EUR in 2020 to 8,734.76 EUR in 2021.\n\nThe substantial increase in staff and running costs are the primary contributors to the overall rise in total expenses from 2020 to 2021.  While the provided data doesn't explain *why* these costs increased, it clearly identifies them as the main drivers.")

Total expenses increased significantly from 821,384.63 EUR in 2020 to 1,744,964.57 EUR in 2021.  A major driver of 
this increase is staff costs, rising from 710,717.99 EUR in 2020 to 1,385,839.57 EUR in 2021.  Further analysis of 
the operating expenses reveals the following changes:                                                              

 • Running costs: Increased from 87,091.87 EUR in 2020 to 350,390.24 EUR in 2021.                                  
 • Recruitment costs: Increased from 18,481.07 EUR in 2020 to an undisclosed amount in 2021 (likely included in    
   staff costs or other operating expenses).                                                                       
 • Finance costs: Increased from 5,093.70 EUR in 2020 to 8,734.76 EUR in 2021.                                     

The substantial increase in staff and running costs are the primary contributors to the overall rise in total      
expenses from 2020 to 2021.  While the provided data doesn't explain why these costs increased, it clearly         
identifies them as the main drivers.

In [106]:
agent_react_full_pdf.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------



---------------------------------------- iteration 2  ----------------------------------------


'The primary drivers behind the significant increase in total expenses in 2021 compared to 2020 are increased staffing and related costs, as well as a substantial rise in spending on studies and technical assistance.  While the COVID-19 pandemic affected some meeting formats, it did not appear to significantly offset the overall increase in expenses.'

* <span style="color: blue; font-weight: bold;"> all three methods find the answer because it is very straigtforward<span>


### question 3

In [107]:
query="Unused budget appropriations decreased from €3,254,632.46 in 2020 to €737,686.05 in 2021. What caused this decrease?"

In [108]:
agent_one_big_prompt.run(query=query)

'Unused budget appropriations decreased from €3,254,632.46 in 2020 to €737,686.05 in 2021 due to the following:\n\n* **Unused appropriations from 2019 and 2020:** €48,692.50 was unused from the 2019 and 2020 budgets.\n* **Unused appropriations from 2021:** €688,993.55 was unused from the 2021 budget.\n\nThe sum of these two amounts (€48,692.50 + €688,993.55) equals the total unused budget appropriations of €737,686.05 in 2021.  The decrease reflects the utilization of prior year appropriations and the lower amount of unused appropriations in the current year.'

In [110]:
rich_Markdown('Unused budget appropriations decreased from €3,254,632.46 in 2020 to €737,686.05 in 2021 due to the following:\n\n* **Unused appropriations from 2019 and 2020:** €48,692.50 was unused from the 2019 and 2020 budgets.\n* **Unused appropriations from 2021:** €688,993.55 was unused from the 2021 budget.\n\nThe sum of these two amounts (€48,692.50 + €688,993.55) equals the total unused budget appropriations of €737,686.05 in 2021.  The decrease reflects the utilization of prior year appropriations and the lower amount of unused appropriations in the current year.')

Unused budget appropriations decreased from €3,254,632.46 in 2020 to €737,686.05 in 2021 due to the following:     

 • Unused appropriations from 2019 and 2020: €48,692.50 was unused from the 2019 and 2020 budgets.                 
 • Unused appropriations from 2021: €688,993.55 was unused from the 2021 budget.                                   

The sum of these two amounts (€48,692.50 + €688,993.55) equals the total unused budget appropriations of           
€737,686.05 in 2021.  The decrease reflects the utilization of prior year appropriations and the lower amount of   
unused appropriations in the current year.

In [111]:
agent_react.run(query=query)


---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------


'The decrease in unused budget appropriations from €3,254,632.46 in 2020 to €737,686.05 in 2021 can be attributed to a combination of factors:\n\n* **Increased utilization of funds:**  The decrease suggests that more of the budgeted funds were actually used for their intended purposes in 2021 compared to 2020.  This is reflected in the revenue adjustments related to investments and legal commitments, which were higher in 2021.\n* **Repayment to Contracting Parties:** Unused appropriations are proposed for repayment to Contracting Parties. While the specific amounts repaid aren\'t detailed here, this is a standard procedure contributing to the decrease.\n* **Carry-forward of contributions:**  The Budget Committee recommended carrying forward unused appropriations belonging to South East Europe parties as contributions to the next year\'s budget. This effectively reduces the amount classified as "unused" at the end of 2021.\n* **Changes in budgeting and spending patterns:** The significa

In [112]:
rich_Markdown('The decrease in unused budget appropriations from €3,254,632.46 in 2020 to €737,686.05 in 2021 can be attributed to a combination of factors:\n\n* **Increased utilization of funds:**  The decrease suggests that more of the budgeted funds were actually used for their intended purposes in 2021 compared to 2020.  This is reflected in the revenue adjustments related to investments and legal commitments, which were higher in 2021.\n* **Repayment to Contracting Parties:** Unused appropriations are proposed for repayment to Contracting Parties. While the specific amounts repaid aren\'t detailed here, this is a standard procedure contributing to the decrease.\n* **Carry-forward of contributions:**  The Budget Committee recommended carrying forward unused appropriations belonging to South East Europe parties as contributions to the next year\'s budget. This effectively reduces the amount classified as "unused" at the end of 2021.\n* **Changes in budgeting and spending patterns:** The significant difference between the two years suggests potential changes in how the budget was allocated and managed.  Further investigation might be needed to understand specific shifts in priorities or spending efficiency.\n\nThe information provided highlights the key factors contributing to the decrease, but a more detailed analysis of the budget reports and auditor\'s notes would be necessary to provide a precise breakdown of the contributing factors.')

The decrease in unused budget appropriations from €3,254,632.46 in 2020 to €737,686.05 in 2021 can be attributed to
a combination of factors:                                                                                          

 • Increased utilization of funds:  The decrease suggests that more of the budgeted funds were actually used for   
   their intended purposes in 2021 compared to 2020.  This is reflected in the revenue adjustments related to      
   investments and legal commitments, which were higher in 2021.                                                   
 • Repayment to Contracting Parties: Unused appropriations are proposed for repayment to Contracting Parties. While
   the specific amounts repaid aren't detailed here, this is a standard procedure contributing to the decrease.    
 • Carry-forward of contributions:  The Budget Committee recommended carrying forward unused appropriations        
   belonging to South East Europe parties as contributions to the next year's budget. This effectively reduces the 
   amount classified as "unused" at the end of 2021.                                                               
 • Changes in budgeting and spending patterns: The significant difference between the two years suggests potential 
   changes in how the budget was allocated and managed.  Further investigation might be needed to understand       
   specific shifts in priorities or spending efficiency.                                                           

The information provided highlights the key factors contributing to the decrease, but a more detailed analysis of  
the budget reports and auditor's notes would be necessary to provide a precise breakdown of the contributing       
factors.

In [113]:
agent_react_full_pdf.run(query=query)



---------------------------------------- iteration 0  ----------------------------------------



---------------------------------------- iteration 1  ----------------------------------------



---------------------------------------- iteration 2  ----------------------------------------


'The decrease in unused budget appropriations from €3,254,632.46 in 2020 to €737,686.05 in 2021 is primarily due to the utilization and repayment of 2019 and 2020 unused appropriations.  The breakdown of the utilization, carry forward, and repayment is detailed in Note 13 of the financial statements.'

* <span style="color: blue; font-weight: bold;"> the information is in Note 5, one big prompt agent failed to find that <span>
* <span style="color: blue; font-weight: bold;"> ReAct with RAG correctly found it<span>
* <span style="color: blue; font-weight: bold;"> ReAct with full pdf did not find the specific reason, which is `increased utilization of funds for legal commitments in 2021`<span>